In [1]:
# Evaluation of experiments
import matplotlib.pyplot as plt
import pandas as pd

dfExperiments45 = pd.read_csv('experiments_similarityScore_45_negativeSamples_3_2019-01-07T18-27-30.049.csv', sep=',', low_memory=False, header=None)
dfExperiments45.columns = ['Timestamp', 'Experiment', 'Class', 'TargetTable', 'Precision', 'Recall', 'F-1', 'Blocker', 'BlockingFunction', 'TrainedOn', 'TrainedModel']
dfExperiments45['Indexing Threshold'] = '0.45'

dfExperiments70 = pd.read_csv('experiments_similarityScore_70_negativeSamples_3_2019-01-07T17-59-12.297.csv', sep=',', low_memory=False, header=None)
dfExperiments70.columns = ['Timestamp', 'Experiment', 'Class', 'TargetTable', 'Precision', 'Recall', 'F-1', 'Blocker', 'BlockingFunction', 'TrainedOn', 'TrainedModel']
dfExperiments70['Indexing Threshold'] = '0.7'

In [3]:
#Keep only baselines
dfExperiments45 = dfExperiments45[~dfExperiments45['Experiment'].str.contains('transfer_')]
dfExperiments70 = dfExperiments70[~dfExperiments70['Experiment'].str.contains('transfer_')]
#display(dfExperiments)

#Compare baselines to matching rule
def compareLearnedRuleWithBaseline(dfExperiments):

    dfBaselineExperimentsCompared = pd.DataFrame()

    for targetTable in dfExperiments['TargetTable'].unique():
        dfSubSetExperiments = dfExperiments[dfExperiments['TargetTable'] == targetTable]
        dfLearnedMatchingRule = dfSubSetExperiments[dfSubSetExperiments['Experiment'].str.contains('learned_')]
        dfBaselinesMatchingRule = dfSubSetExperiments[dfSubSetExperiments['Experiment'].str.contains('baseline_')]
    
        for indexLearned, rowLearned in dfLearnedMatchingRule.iterrows():
            dfBaselineExperimentsTmpCompared = pd.DataFrame()
            for indexBaseline, rowBaseline in dfBaselinesMatchingRule.iterrows():
                newRow = rowLearned.copy()
                newRow['Precision-Delta'] = rowLearned['Precision'] - rowBaseline['Precision']
                newRow['Recall-Delta'] = rowLearned['Recall'] - rowBaseline['Recall']
                newRow['F-1-Delta'] = rowLearned['F-1'] - rowBaseline['F-1']
                newRow['Experiment'] = rowBaseline['Experiment']
            
                dfBaselineExperimentsTmpCompared = dfBaselineExperimentsTmpCompared.append(newRow, ignore_index=True)
            
            dfBaselineExperimentsTmpCompared = dfBaselineExperimentsTmpCompared.sort_values(by=['F-1-Delta']).iloc[0]
            dfBaselineExperimentsCompared = dfBaselineExperimentsCompared.append(dfBaselineExperimentsTmpCompared)
    return dfBaselineExperimentsCompared

dfBaselineExperimentsCompared = compareLearnedRuleWithBaseline(dfExperiments45)
dfBaselineExperimentsCompared = dfBaselineExperimentsCompared.append(compareLearnedRuleWithBaseline(dfExperiments70))
dfBaselineExperimentsCompared = dfBaselineExperimentsCompared.sort_values(by=['TargetTable'])

dfBaselineExperimentsCompared = dfBaselineExperimentsCompared.filter(items=['TargetTable', 'Indexing Threshold',  'TrainedModel', 'Precision', 'Recall',  'F-1', 'Experiment','Precision-Delta',  'Recall-Delta','F-1-Delta'])
dfBaselineExperimentsCompared['Experiment'] = dfBaselineExperimentsCompared['Experiment'].str.replace('baseline_', '').str.replace('_city', '')
dfBaselineExperimentsCompared['Experiment'] = dfBaselineExperimentsCompared['Experiment'].str.split('-', expand=True)[0]
dfBaselineExperimentsCompared['TrainedModel'] = dfBaselineExperimentsCompared['TrainedModel'].str.replace('J48', 'DT')
dfBaselineExperimentsCompared['TrainedModel'] = dfBaselineExperimentsCompared['TrainedModel'].str.replace('SimpleLogistic', 'LR')
dfBaselineExperimentsCompared['TargetTable'] = dfBaselineExperimentsCompared['TargetTable'].str.replace('.csv', '')
dfBaselineExperimentsCompared['TargetTable'] = dfBaselineExperimentsCompared['TargetTable'].str.replace('GeonamesData', 'GN')
dfBaselineExperimentsCompared['TargetTable'] = dfBaselineExperimentsCompared['TargetTable'].str.replace('WebTableDataCity', 'WT')
dfBaselineExperimentsCompared['TargetTable'] = dfBaselineExperimentsCompared['TargetTable'].str.replace('WikiDataCity', 'WD')

dfBaselineExperimentsCompared = dfBaselineExperimentsCompared.replace('all', 'All')
dfBaselineExperimentsCompared = dfBaselineExperimentsCompared.replace('bag_of_words', 'BoW')
dfBaselineExperimentsCompared = dfBaselineExperimentsCompared.replace('linear_combination', 'LC')
dfBaselineExperimentsCompared = dfBaselineExperimentsCompared.replace('label', 'Label')
dfBaselineExperimentsCompared = dfBaselineExperimentsCompared.round(3)
display(dfBaselineExperimentsCompared)

print(dfBaselineExperimentsCompared.to_latex(index=False))

,TargetTable,Indexing Threshold,TrainedModel,Precision,Recall,F-1,Experiment,Precision-Delta,Recall-Delta,F-1-Delta
1,GN,0.45,DT,0.978,0.996,0.987,BoW,0.212,0.001,0.121
1,GN,0.45,LR,0.963,0.995,0.979,BoW,0.196,-0.001,0.112
1,GN,0.7,DT,0.995,0.991,0.993,BoW,0.228,-0.005,0.126
1,GN,0.7,LR,0.977,0.995,0.986,BoW,0.210,-0.001,0.119
0,WT,0.45,DT,0.584,0.945,0.722,Label,-0.002,0.145,0.045
0,WT,0.45,LR,0.546,0.964,0.697,Label,-0.040,0.164,0.020
0,WT,0.7,DT,0.571,0.945,0.712,Label,-0.015,0.145,0.035
0,WT,0.7,LR,0.536,0.945,0.684,Label,-0.051,0.145,0.007
1,WD,0.45,DT,0.992,0.996,0.994,BoW,0.268,-0.004,0.154
1,WD,0.45,LR,0.973,0.999,0.986,BoW,0.249,-0.001,0.146


\begin{tabular}{lllrrrlrrr}
\toprule
TargetTable & Indexing Threshold & TrainedModel &  Precision &  Recall &    F-1 & Experiment &  Precision-Delta &  Recall-Delta &  F-1-Delta \\
\midrule
         GN &               0.45 &           DT &      0.978 &   0.996 &  0.987 &        BoW &            0.212 &         0.001 &      0.121 \\
         GN &               0.45 &           LR &      0.963 &   0.995 &  0.979 &        BoW &            0.196 &        -0.001 &      0.112 \\
         GN &                0.7 &           DT &      0.995 &   0.991 &  0.993 &        BoW &            0.228 &        -0.005 &      0.126 \\
         GN &                0.7 &           LR &      0.977 &   0.995 &  0.986 &        BoW &            0.210 &        -0.001 &      0.119 \\
         WT &               0.45 &           DT &      0.584 &   0.945 &  0.722 &      Label &           -0.002 &         0.145 &      0.045 \\
         WT &               0.45 &           LR &      0.546 &   0.964 &  0.697 &      Lab

In [5]:
import csv

dfExperimentsAll = dfBaselineExperimentsCompared[dfBaselineExperimentsCompared['TargetTable'].str.contains('All')]
dfExperimentsOther = dfBaselineExperimentsCompared[~dfBaselineExperimentsCompared['TargetTable'].str.contains('All')]

dfAggregatedExperiments = dfExperimentsOther.groupby(['Indexing Threshold']).mean()
dfAggregatedExperiments['Level'] = 'Other'
dfAggregatedExperimentsAll = dfExperimentsAll.groupby(['Indexing Threshold']).mean()
dfAggregatedExperimentsAll['Level'] = 'All'

dfAggregatedExperiments = dfAggregatedExperiments.append(dfAggregatedExperimentsAll)
display(dfAggregatedExperiments)
dfAggregatedExperiments.to_csv('rq1_aggregated_results_city.csv', sep=',', encoding='utf-8', quotechar='"', quoting=csv.QUOTE_ALL)

,Precision,Recall,F-1,Precision-Delta,Recall-Delta,F-1-Delta,Level
Indexing Threshold,,,,,,,
0.45,0.839333,0.982500,0.894167,0.147167,0.050667,0.099667,Other
0.7,0.844167,0.978167,0.894167,0.151500,0.046167,0.099333,Other
0.45,0.783500,0.973000,0.868000,-0.015500,0.300000,0.137000,All
0.7,0.800000,0.964000,0.874000,0.001000,0.291000,0.143500,All


In [ ]:
#Rearrange DataFrame
def rearrange(df):
    dfAllExperimentsRearranged = pd.DataFrame()
    for index, row in df.iterrows():
        rowPrecision = row
        rowPrecision['ResultValue'] = row['Precision-Delta']
        rowPrecision['ResultValueType'] = 'Precision-Delta'
        
        dfAllExperimentsRearranged = dfAllExperimentsRearranged.append(rowPrecision)
        
        rowRecall = row
        rowRecall['ResultValue'] = row['Recall-Delta']
        rowRecall['ResultValueType'] = 'Recall-Delta'
        
        dfAllExperimentsRearranged = dfAllExperimentsRearranged.append(rowPrecision)
        
        #rowF1 = row
        #rowF1['ResultValue'] = row['F-1']
        #rowF1['ResultValueType'] = 'F-1'
        
        #dfAllExperimentsRearranged = dfAllExperimentsRearranged.append(rowF1)
    
    return dfAllExperimentsRearranged

dfAllExperimentsRearranged = rearrange(dfBaselineExperimentsCompared)

display(dfAllExperimentsRearranged)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plotResults(df, title):
    sns.set(style="whitegrid")
    fig, ax1 = plt.subplots(figsize=(20,8))
    sns.catplot(x="shortExperiment", y="ResultValue", hue="ResultValueType", data=df, height=6, kind="bar", ax=ax1)
    
    titleAll = title
    ax1.set_title(titleAll, fontweight="bold", size=20) # Title
    ax1.set_xlabel("Experiments",fontsize = 20.0)
    ax1.set_ylabel("" ,fontsize = 20.0) # Y label
    ax1.tick_params(direction='out', length=6, width=2, labelsize=20, grid_alpha=0.5)
    ax1.tick_params(axis = 'x', labelrotation=45)
    ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=20)


dfAllExperimentsRearranged['shortExperiment'] = dfAllExperimentsRearranged['TrainedModel'].str.cat(dfAllExperimentsRearranged['Experiment'].str.replace('_author', '').str.replace('baseline_', '').str.replace('hotels_www.',''), sep='-')
plotResults(dfAllExperimentsRearranged, 'Baseline Comparison')

In [ ]:
display(dfAllExperimentsRearranged.filter(items=['shortExperiment','Precision', 'Recall', 'F-1']).drop_duplicates(keep='first'))

In [ ]:
round(dfAllExperimentsRearranged['ResultValue'].mean(),4)